In [1]:
import sys
import os
import logging
from datetime import datetime, date
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
from sqlalchemy import create_engine
import json
from dotenv import load_dotenv

# プロジェクトのルートディレクトリをパスに追加
sys.path.append('..')

# ログ設定
logging.basicConfig(
    level=logging.WARNING, # WARNING以上のみ表示（INFO, DEBUGを非表示）
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# 環境変数読み込み
load_dotenv()

print("=== 競馬レースデータ収集システム ===")
print(f"実行時刻: {datetime.now()}")



=== 競馬レースデータ収集システム ===
実行時刻: 2025-09-01 20:20:06.029243


In [2]:
# 必要なモジュールをインポート
from src.scraping.scrapers.race_scraper import RaceScraper
from src.scraping.scrapers.horse_scraper import HorseScraper
from src.database.schemas.race_schema import Race, RaceResult
from src.scraping.storage.postgresql_storage import PostgreSQLStorage

print("✅ システム初期化完了")

✅ システム初期化完了


In [3]:
# =============================================================================
# 1. 接続設定
# =============================================================================

print("🔌 データベース接続を設定中...")

# PostgreSQL接続設定
pg_config = {
    'host': os.getenv('POSTGRES_HOST', 'localhost'),
    'database': os.getenv('POSTGRES_DB', 'stallion_db'),
    'user': os.getenv('POSTGRES_USER', 'stallion_user'),
    'password': os.getenv('POSTGRES_PASSWORD'),
    'port': int(os.getenv('POSTGRES_PORT', '5432'))
}

# SQLAlchemy エンジン作成（pandasで使用）
DATABASE_URL = f"postgresql://{pg_config['user']}:{pg_config['password']}@{pg_config['host']}:{pg_config['port']}/{pg_config['database']}"
engine = create_engine(DATABASE_URL)

print("✅ 接続設定完了")

🔌 データベース接続を設定中...
✅ 接続設定完了


In [4]:
# =============================================================================
# pandas.read_sql
# =============================================================================

def query(sql):
    """SQLクエリを実行してDataFrameで結果を返す"""
    return pd.read_sql(sql, engine)

def show_tables():
    """全テーブル一覧を表示"""
    sql = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
    return query(sql)

def desc(table_name):
    """テーブル構造を表示"""
    sql = f"""
    SELECT column_name, data_type, is_nullable, column_default 
    FROM information_schema.columns 
    WHERE table_name = '{table_name}' 
    ORDER BY ordinal_position;
    """
    return query(sql)

def count_all(table_name):
    """テーブルの行数を取得"""
    sql = f"SELECT COUNT(*) as count FROM {table_name};"
    return query(sql).iloc[0, 0]

In [17]:
# 仮で登録したhorseデータを全て取得

print("📋 全テーブル一覧")

df = query("""
        SELECT
            id,
            name_ja
        FROM horses
        WHERE profile IS NULL
        LIMIT 100;

    """)
print(df)

horse_list = df.to_dict('records')
# print(horse_list)

📋 全テーブル一覧
             id    name_ja
0    2011104344  ハギノハイブリッド
1    2011102556  メイショウスミトモ
2    2016106126     ソルトイブキ
3    2013102679    ロードクエスト
4    2015101654  アフリカンゴールド
..          ...        ...
295  2014105887    サトノアーサー
296  2014110081   ベストアプローチ
297  2014105517   スティッフェリオ
298  2014106132  トリコロールブルー
299  2014100534    ウインガナドル

[300 rows x 2 columns]


In [9]:
# 取得された馬に対して、 horsesと horse_relationsを登録

horse_scraper = HorseScraper(delay=1.0)
horse_storage = PostgreSQLStorage()

for horse in horse_list:
    horse_id = horse['id']
    result = horse_scraper.scrape_horse_detail(horse_id=horse_id)

    # スクレイピング成功時のみ保存
    if result:
        horse_data = result[0]
        relations = result[1]

        horse_storage.insert_horse_full(horse_data)
        horse_storage.insert_horse_relations(relations)
        print(f"✅ 馬情報保存完了: {horse['name_ja']} (ID: {horse_id})")
    else:
        print(f"❌ 馬情報のスクレイピングに失敗: ID {horse_id}")



{'id': '2011104344', 'name_ja': 'ハギノハイブリッド'}
🔍 ページ取得: https://db.netkeiba.com/horse/2011104051/
🔍 馬詳細ページ取得: 2011104051
    📄 ページタイトル: ワールドインパクト (World Impact) | 競走馬データ - netkeiba
    📝 馬名取得: ワールドインパクト
    🔤 性別取得: stallion
    🔤 英語名取得: World Impact
    ✅ プロフィールテーブル発見
      - 生年月日 (birth_date): 2011-04-05
      - 調教師 (trainer): 友道康夫(栗東)
      - 馬主 (owner): シルクレーシング
      - 募集情報 (offering_info): {'price_per_unit': 9, 'total_units': 500, 'raw_text': '1口:9万円/500口'}
      - 生産者 (breeder): ノーザンファーム
      - 産地 (birthplace): 安平町
      - 獲得賞金 (中央) (total_prize_central): 5818
      - 通算成績 (career_record): {'starts': 11, 'wins': 2, 'win_rate': 18.2, 'first': 2, 'second': 5, 'third': 0, 'others': 4}
      - 近親馬 (related_horses): トリコロールブルー、ダノンジェラート
  🌳 血統関係取得: 2011104051
    🧬 父ID設定: 2002100816
    🧬 母ID設定: 000a010b6f
    🧬 母父ID設定: 000a002328
    💕 種付関係新規作成: 2002100816 × 000a010b6f → 子供: 2011104051
    📋 取得データ項目数: 16
  ✅ 基本情報取得成功: ワールドインパクト
